# Döntési fa woe


## Adatok betöltése

In [259]:

import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_bc050f45180b4c1bb8886542b85a43fe = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='6akYbYHItNWmuiZPpgS8f0DnZNTVPSN3hYgF_EVLfHhd',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_bc050f45180b4c1bb8886542b85a43fe.get_object(Bucket='szakdolgozat-donotdelete-pr-nxru0g4tgksnkv',Key='WOE_train.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

train_input = pd.read_csv(body, sep=';',decimal=',')
train_input.head(10)


,DPD90_12m,customer_type,product_type,onero,age_of_asset (honap),sales_channel,Nace risk tier
0,0,-0.029802,-0.180531,0.010955,0.603264,0.232891,0.585450
1,0,-0.029802,-0.180531,-0.210588,-1.386171,-0.883030,-0.893508
2,0,-0.029802,-0.180531,-0.210588,-0.365448,0.232891,-0.224232
3,0,-0.029802,-0.180531,0.728017,0.603264,0.232891,-0.224232
4,0,-0.029802,-0.180531,-0.210588,0.603264,0.232891,-0.224232
5,0,-0.029802,-0.180531,-0.210588,-0.365448,-0.883030,0.585450
6,0,0.276928,-0.180531,0.728017,0.603264,0.232891,0.585450
7,0,-0.029802,-0.180531,0.010955,0.603264,0.232891,0.585450
8,0,-0.029802,-0.180531,-0.210588,-0.365448,0.232891,-0.224232
9,0,-0.029802,-0.180531,-0.210588,-0.365448,0.232891,0.585450


In [260]:

body = client_bc050f45180b4c1bb8886542b85a43fe.get_object(Bucket='szakdolgozat-donotdelete-pr-nxru0g4tgksnkv',Key='WOE_test.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

test_input = pd.read_csv(body, sep=';',decimal=',')
test_input.head()


,DPD90_12m,customer_type,product_type,onero,age_of_asset (honap),sales_channel,Nace risk tier
0,0,-0.029802,-0.180531,0.010955,0.603264,0.232891,0.585450
1,0,-0.029802,-0.180531,-0.210588,-1.386171,-0.883030,0.585450
2,0,-0.029802,-0.180531,0.728017,0.603264,0.232891,0.585450
3,0,-0.029802,-0.180531,0.728017,0.603264,0.232891,0.585450
4,0,-0.029802,-0.180531,-0.210588,0.603264,0.232891,-0.893508


In [261]:
train_input.shape

(7339, 7)

In [262]:
test_input.shape

(3083, 7)

## Adatok előkészítése

In [263]:
import numpy as np

In [264]:
X_train = np.asarray(train_input[['onero', 'age_of_asset (honap)','sales_channel', 'Nace risk tier']])
X_train[0:1]

array([[0.01095466, 0.6032641 , 0.23289082, 0.58545026]])

In [265]:
X_test = np.asarray(test_input[['onero', 'age_of_asset (honap)','sales_channel', 'Nace risk tier']])
X_test[0:1]

array([[0.01095466, 0.6032641 , 0.23289082, 0.58545026]])

In [266]:
y_train = np.asarray(train_input['DPD90_12m'])
y_train [0:5]

array([0, 0, 0, 0, 0])

In [267]:
y_test = np.asarray(test_input['DPD90_12m'])
y_test [0:5]

array([0, 0, 0, 0, 0])

## Modellezés

In [282]:
from sklearn.tree import DecisionTreeClassifier
LR = DecisionTreeClassifier(criterion="entropy", max_depth = 50, min_samples_split=2).fit(X_train,y_train)
LR

DecisionTreeClassifier(criterion='entropy', max_depth=50)

In [283]:
yhat_test = LR.predict_proba(X_test)
yhat_test

array([[0.99677766, 0.00322234],
       [0.9516129 , 0.0483871 ],
       [1.        , 0.        ],
       ...,
       [1.        , 0.        ],
       [0.78787879, 0.21212121],
       [0.9908953 , 0.0091047 ]])

In [284]:
yhat_train = LR.predict_proba(X_train)
yhat_train

array([[0.99677766, 0.00322234],
       [0.88888889, 0.11111111],
       [0.97354497, 0.02645503],
       ...,
       [0.99677766, 0.00322234],
       [1.        , 0.        ],
       [0.92105263, 0.07894737]])

In [285]:
tech, y_predicted_train = zip(*yhat_train)
y_predicted_train[0:5]

(0.00322234156820623,
 0.1111111111111111,
 0.026455026455026454,
 0.011594202898550725,
 0.0199203187250996)

In [286]:
tech, y_predicted_test = zip(*yhat_test)
y_predicted_test[0:5]

(0.00322234156820623, 0.04838709677419355, 0.0, 0.0, 0.048859934853420196)

## Modell ereje

In [287]:
from sklearn.metrics import roc_auc_score
roc=roc_auc_score(y_train, y_predicted_train)
gini=2*roc-1
gini

0.5930108311405882

In [288]:
from sklearn.metrics import roc_auc_score
roc=roc_auc_score(y_test, y_predicted_test)
gini=2*roc-1
gini

0.45223838038209285